# Churn Analytics — Cleaning, Feature Engineering, and EDA (Executed)
This lightweight notebook is executed and includes textual outputs and a small preview to keep it fast in your environment.
Outputs (CSVs, plots) are saved under `outfile/`.

In [1]:

import os, numpy as np, pandas as pd, matplotlib.pyplot as plt

OUTDIR = "/mnt/data/outfile"
PLOTDIR = os.path.join(OUTDIR, "plots")
os.makedirs(OUTDIR, exist_ok=True)
os.makedirs(PLOTDIR, exist_ok=True)

def standardize_columns(df):
    df = df.copy()
    df.columns = (
        df.columns.str.strip()
        .str.lower()
        .str.replace(r"[^a-z0-9]+", "_", regex=True)
        .str.replace(r"_+", "_", regex=True)
        .str.strip("_")
    )
    return df

SYNONYMS = {
    "customer_id": ["customer_id", "customerid", "user_id", "userid", "id"],
    "date": ["date", "event_date", "activity_date", "txn_date", "timestamp"],
    "login": ["login", "logins", "login_count", "active_flag", "is_active"],
    "watchtimeminutes": ["watchtimeminutes", "watch_time_minutes", "watch_time", "mins_watched", "view_minutes"],
    "contentpreference": ["contentpreference", "content_pref", "genre_preference", "preferred_genre"],
    "subscriptionplan": ["subscriptionplan", "plan", "tier", "package"],
    "agegroup": ["agegroup", "age_group", "age_bucket"],
    "churned": ["churned", "is_churned", "churn"],
    "churndate": ["churndate", "churn_date"],
    "location": ["location", "city", "region"],
    "householdsize": ["householdsize", "household_size", "hh_size"],
    "primarydevice": ["primarydevice", "device", "device_type"],
    "billingcycle": ["billingcycle", "billing_cycle", "cycle"],
    "isautorenew": ["isautorenew", "auto_renew", "autorenew", "auto_renewal"],
}

def resolve_col(df, key):
    cols = list(df.columns)
    for cand in SYNONYMS.get(key, []):
        if cand in cols:
            return cand
    return None

def mode_or_nan(series):
    if series is None or series.empty:
        return np.nan
    counts = series.value_counts(dropna=True)
    return counts.idxmax() if not counts.empty else np.nan

def safe_to_datetime(s):
    return pd.to_datetime(s, errors="coerce") if s is not None else s

# Load
churn_df = pd.read_excel("/mnt/data/customer_churn_data.xlsx")
demo_df  = pd.read_csv("/mnt/data/demographic_data.csv")
subs_df  = pd.read_csv("/mnt/data/subscription_payment_history.csv")

churn_df = standardize_columns(churn_df)
demo_df  = standardize_columns(demo_df)
subs_df  = standardize_columns(subs_df)

# Align IDs
for df in [churn_df, demo_df, subs_df]:
    cid = resolve_col(df, "customer_id")
    if cid != "customer_id":
        df.rename(columns={cid: "customer_id"}, inplace=True)

# Align churn log columns
date_col = resolve_col(churn_df, "date")
if date_col and date_col != "date": churn_df.rename(columns={date_col: "date"}, inplace=True)
churn_df["date"] = safe_to_datetime(churn_df["date"]) if "date" in churn_df.columns else pd.NaT
for k in ["login","watchtimeminutes","contentpreference","subscriptionplan","agegroup","churned","churndate"]:
    c = resolve_col(churn_df, k)
    if c and c != k: churn_df.rename(columns={c: k}, inplace=True)
if "churndate" in churn_df.columns: churn_df["churndate"] = safe_to_datetime(churn_df["churndate"])

# Align others
for k in ["location","householdsize","primarydevice","agegroup"]:
    c = resolve_col(demo_df, k)
    if c and c != k: demo_df.rename(columns={c: k}, inplace=True)
for k in ["billingcycle","isautorenew"]:
    c = resolve_col(subs_df, k)
    if c and c != k: subs_df.rename(columns={c: k}, inplace=True)

# Feature engineering
grouped = churn_df.groupby("customer_id")
def gmin(col):   return grouped[col].min()   if col in churn_df.columns else pd.Series(dtype="datetime64[ns]")
def gmax(col):   return grouped[col].max()   if col in churn_df.columns else pd.Series(dtype="datetime64[ns]")
def gsum(col):   return grouped[col].sum()   if col in churn_df.columns else pd.Series(dtype="float")
def gmean(col):  return grouped[col].mean()  if col in churn_df.columns else pd.Series(dtype="float")
def gunique(col):return grouped[col].nunique() if col in churn_df.columns else pd.Series(dtype="float")
def gmode(col):  return grouped[col].apply(mode_or_nan) if col in churn_df.columns else pd.Series(dtype="object")

global_max_date = churn_df["date"].max() if "date" in churn_df.columns else pd.NaT

feat = pd.DataFrame({
    "first_activity": gmin("date"),
    "last_activity":  gmax("date"),
    "days_observed":  gunique("date"),
    "total_logins":   gsum("login"),
    "login_rate":     gmean("login"),
    "total_watch_minutes": gsum("watchtimeminutes"),
    "avg_watch_per_day":   gmean("watchtimeminutes"),
    "preferred_content":   gmode("contentpreference"),
    "plan_mode_activity":  gmode("subscriptionplan"),
    "age_group_mode":      gmode("agegroup"),
})

if "churned" in churn_df.columns:
    feat["churned_ever"] = grouped["churned"].max().astype(bool)
elif "churndate" in churn_df.columns:
    feat["churned_ever"] = grouped["churndate"].apply(lambda s: s.notna().any()).astype(bool)
else:
    feat["churned_ever"] = False

if "churndate" in churn_df.columns:
    feat["churn_date_first"] = grouped["churndate"].apply(lambda s: s.dropna().min() if s.notna().any() else pd.NaT)
else:
    feat["churn_date_first"] = pd.NaT

if "last_activity" in feat.columns and pd.notna(global_max_date):
    feat["recency_days"] = (global_max_date - feat["last_activity"]).dt.days
else:
    feat["recency_days"] = np.nan

feat = feat.reset_index()

master = feat.merge(demo_df, on="customer_id", how="left", suffixes=("","_demo"))
keep_cols = ["customer_id"]
if "billingcycle" in subs_df.columns: keep_cols.append("billingcycle")
if "isautorenew" in subs_df.columns: keep_cols.append("isautorenew")
subs_slim = subs_df[keep_cols].copy()
master = master.merge(subs_slim, on="customer_id", how="left", suffixes=("","_subs"))

master["active_days"] = master["total_logins"].fillna(0) if "total_logins" in master.columns else np.nan
def _avg_watch_per_active(r):
    try:
        num = float(r.get("total_watch_minutes", 0) if not pd.isna(r.get("total_watch_minutes", np.nan)) else 0)
        den = float(r.get("active_days", 0) if not pd.isna(r.get("active_days", np.nan)) else 0)
        return (num / den) if den > 0 else 0.0
    except Exception:
        return 0.0
master["avg_watch_per_active_day"] = master.apply(_avg_watch_per_active, axis=1)
master["tenure_days"] = (master["last_activity"] - master["first_activity"]).dt.days + 1 if "last_activity" in master.columns and "first_activity" in master.columns else np.nan

master.rename(columns={
    "plan_mode_activity": "PlanMostSeen",
    "age_group_mode": "AgeGroup",
    "preferred_content": "ContentPreference",
    "location": "Location",
    "householdsize": "HouseholdSize",
    "primarydevice": "PrimaryDevice",
    "billingcycle": "BillingCycle",
    "isautorenew": "IsAutoRenew",
    "customer_id": "CustomerID"
}, inplace=True)

for col in ["PlanMostSeen","AgeGroup","ContentPreference","Location","PrimaryDevice","BillingCycle"]:
    if col in master.columns:
        master[col] = master[col].astype("category")

master["churn_label"] = master["churned_ever"].astype(int)

cleaned_path = os.path.join(OUTDIR, "cleaned_master.csv")
master.to_csv(cleaned_path, index=False)

cat_cols = [c for c in ["PlanMostSeen","AgeGroup","ContentPreference","Location","PrimaryDevice","BillingCycle"] if c in master.columns]
num_cols = [c for c in [
    "days_observed","total_logins","login_rate","total_watch_minutes","avg_watch_per_day",
    "recency_days","active_days","avg_watch_per_active_day","tenure_days","HouseholdSize"
] if c in master.columns]

ml_df = master[["CustomerID","churn_label"] + cat_cols + num_cols].copy()
ml_df = pd.get_dummies(ml_df, columns=cat_cols, dummy_na=True)

for c in num_cols:
    mu = ml_df[c].mean()
    sd = ml_df[c].std(ddof=0)
    ml_df[c] = 0.0 if (pd.isna(sd) or sd == 0) else (ml_df[c] - mu) / sd

ml_ready_path = os.path.join(OUTDIR, "ml_ready.csv")
ml_df.to_csv(ml_ready_path, index=False)

print("Master shape:", master.shape)
print("ML-ready shape:", ml_df.shape)
print("Saved:", cleaned_path, "and", ml_ready_path)
print("Master preview:")
print(master.head(5))


Master shape: (250, 23)
ML-ready shape: (250, 41)
Saved: /mnt/data/outfile/cleaned_master.csv and /mnt/data/outfile/ml_ready.csv
Master preview:
  CustomerID first_activity last_activity  days_observed  total_logins  \
0    CUST001     2025-04-01    2025-05-22             52            30   
1    CUST002     2025-04-01    2025-06-30             91            46   
2    CUST003     2025-04-01    2025-06-30             91            42   
3    CUST004     2025-04-01    2025-06-30             91            68   
4    CUST005     2025-04-01    2025-06-30             91            65   

   login_rate  total_watch_minutes  avg_watch_per_day ContentPreference  \
0    0.576923                 1114          21.423077            Sports   
1    0.505495                 1635          17.967033     Documentaries   
2    0.461538                 1393          15.307692            Movies   
3    0.747253                 5328          58.549451            Sports   
4    0.714286                 8123 